# 0. Library

In [1]:
import os, time, datetime
os.environ["TOKENIZERS_PARALLELISM"] = "false"
from tqdm import tqdm
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:100% !important;}</style>"))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM


# 1. Config

In [2]:
class Config:
    data_name = "data/dialogue.csv"
    bos_token = '<s>'
    eos_token = '</s>'
    usr_token = '<usr>'
    pad_token = '<pad>'
    sys_token = '<sys>'
    unk_token = '<unk>'
    mask_token = '<mask>'
    max_length = 2 ** 8
    batch_size = 2 ** 3
    epochs = 2 ** 2
    pretrained_model_name = "skt/kogpt2-base-v2"
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    learning_rate = 3e-5
    model_name = 'model.pt'


# 2. Data

In [3]:
raw_data = pd.read_csv(Config.data_name)

In [4]:
N = len(raw_data)
data = raw_data.sample(N)
data

,conversation
536,<usr> 재훈재훈은자기가 좋아하는거 몰라고지식해내가 얘네보고드라마 쓴거아녀 <sy...
192976,<usr> 오늘 광역버스를 타는데 아저씨가 정류장을 지나치려고 하는거야.<sys> ...
176009,<usr> 오늘 바다 놀러 갔을 때 어땠어?<sys> 오랜만에 바다를 가니 좋았었어...
161645,<usr> 안녕 친구요즘 매일 매일이 너무 바쁘다 ㅋㅋ<sys> 너 항상 바빴잖아 ...
113977,<usr> 전세보는거엿우? <sys> 웅웅월세는 돈이넘나많이나감ㅠㅠ <usr> 전세...
...,...
7008,<usr> 아 127 nonstop 노래 좋네내스타일이다 <sys> 그거약간세상을 ...
76337,<usr> 난 어제 11시에 자서 9시반까지 쭉 자는 기염을 토해쩌 상쾌해그리고 기...
130734,<usr> 12월부터우리 1월말에 보겠네미리 준비하자 <sys> 짝짝좋아야어떤 옷을...
57081,<usr> 나지금 보험뭐할거있어서동탄잠깐갔는데초등학생들 어학원 <sys> ㅋㅋ <u...


In [5]:
def valid_size(train_X):
    length = len(train_X)
    return round(0.14 * (1 + length / 10 ** 4) ** (10 ** 4 / length) - 0.13, 2)

In [6]:
data_train, data_valid = train_test_split(data, test_size=valid_size(data), shuffle=True)
data_train.reset_index(inplace=True)
data_valid.reset_index(inplace=True)

In [7]:
data_train

,index,conversation
0,189836,<usr> 아 왔을때 거기안갔다<sys> 어디<usr> 뼈해장국집<sys> 아 맞네...
1,126293,<usr> 내일 길병원 가지 마엄니랑 아버지랑 간다니까. 정지혜 갈 필요 없어 <s...
2,113732,<usr> 언니터틀넥긴거 입을까짧은거 입을까 <sys> 긴거와 짧은거가어디가 짧은건...
3,72613,<usr> 김밥ㅋ사간당 <sys> ㅋㅋㅋㅋ한시간이나 남았노 <usr> 김밥 두줄사갈...
4,27158,<usr> 나는 고구마나 유산균언제쯤 가지러갈수잇을까용 <sys> 언제되냐? <us...
...,...,...
188635,96372,<usr> 근데나오늘책상 옮기고 파티션 만다는거회의실 세팅다잡아놔서오후엔 또 그거해...
188636,26532,<usr> 박홍민야클래스돼? <sys> ㅋㅋ 안되는듯개거지같넹대기 중이랰ㅋㅋ <us...
188637,120865,<usr> 드디어 폰바꾼다 <sys> 야호! <usr> 신났찌 <sys> xs로 바...
188638,40066,<usr> 윤은진얌 약값도 보험비 청구 됑? <sys> 만원이상일 때만그런데 대부분...


In [8]:
data_valid

,index,conversation
0,56193,<usr> 잘대구있니나는팀미팅좀해쓰ㅋㅋ앞으로머연구할지 <sys> 웅웅 ㅋㅋ미팅잘햇오...
1,185961,<usr> 너네회사도 동료들이랑 대화 많이해?<sys> 뭐 커피마시면서 수다떠는정도...
2,35072,<usr> 목상태가 최악을 향해 달려가고있습니다 부릉부릉 <sys> 무슨일이일어나고...
3,31082,<usr> 그외에 다른사람들하고는 잘지내구걍 저번에일땜에 한번 버럭한거밖에없는뎈ㅋ나...
4,51354,<usr> 근데 문제가 생겼어포토존이 좀 짝네 <sys> 엥 그게 뭐얌 ?포토존도 ...
...,...,...
5830,132619,<usr> 나 월요일날 못만날수도 <sys> 왜?무슨일 있어? <usr> 아빠랑 찬...
5831,89745,<usr> 다른거 누르면 나올걸그래도 우선 티빙봐 <sys> 너 봤어? <usr> ...
5832,130339,<usr> 나랑은직관언제가줘? <sys> 날잡아 <usr> 잡아줘 <sys> 언제갈...
5833,58577,<usr> 이번에 갑자기 5명늘었어무서워ㅠㅠ <sys> 하진짜 다단계 교회 이런게 ...


# 3. Tokenizer

In [9]:
tokenizer = AutoTokenizer.from_pretrained(Config.pretrained_model_name,
            bos_token=Config.bos_token, eos_token=Config.eos_token,
            unk_token=Config.unk_token, pad_token=Config.pad_token,
            mask_token=Config.mask_token, model_max_length=Config.max_length)

In [10]:
for i in range(10):
    print(tokenizer.convert_ids_to_tokens(i), end=' ')

<s> </s> <usr> <pad> <sys> <unk> <mask> <d> </d> <unused0> 

# 4. Dataloader

In [23]:
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, Config):
        self.data = data
        self.tokenizer = tokenizer
        self.bos_token = Config.bos_token
        self.eos_token = Config.eos_token
        self.usr_token = Config.usr_token
        self.pad_token = Config.pad_token
        self.sys_token = Config.sys_token
        self.max_length = Config.max_length
        
    def __len__(self):
        return len(self.data)
        
    def __getitem__(self, idx):
        sentence = self.data['conversation'][idx]
        # input_id
        input_id = self.tokenizer.encode(self.bos_token + sentence + self.eos_token)
        # token_type_id
        token_type = []
        loop = True
        for token_id in input_id:
            token = self.tokenizer.convert_ids_to_tokens(token_id)
            
            if token == self.usr_token: loop=True
            elif token == self.sys_token: loop=False
                
            if loop:
                token_type.append(self.usr_token)
            else:
                token_type.append(self.sys_token)
        token_type_id = self.tokenizer.convert_tokens_to_ids(token_type)
        # label
#         start_idx = len(input_id) - \
#             list(reversed(input_id)).index(self.tokenizer.convert_tokens_to_ids(self.sys_token))
#         label = [-100] * start_idx + input_id[start_idx: ]
        # padding
        input_id, token_type_id, label = self.make_padding(input_id, token_type_id, input_id)
        
        return input_id, token_type_id, input_id

    def make_padding(self, input_id, token_type_id, label):
        left_length = self.max_length - len(input_id)
        input_id += [self.tokenizer.pad_token_id] * left_length
        token_type_id += [self.tokenizer.pad_token_id] * left_length
#         label += [-100] * left_length
        
        return input_id, token_type_id, input_id

In [24]:
train_set = CustomDataset(data_train, tokenizer, Config)

In [26]:
input_id, token_type_id, label = train_set[0]
print("input_id", input_id, sep='\n')
print("token_type_id", token_type_id, sep='\n')
print("label", label, sep='\n')

input_id
[0, 2, 9050, 12583, 8137, 7312, 41732, 7967, 6834, 7182, 4, 13400, 2, 13219, 8711, 46812, 8270, 4, 9050, 9622, 7098, 12829, 7060, 9028, 9436, 24848, 9098, 15010, 2, 30401, 13134, 9114, 23775, 9220, 7789, 27450, 31369, 9117, 7703, 7788, 9078, 8702, 4, 25942, 9034, 8137, 11242, 31522, 2, 31416, 30401, 8024, 9034, 8137, 49542, 8263, 4, 13219, 8711, 46812, 9784, 8133, 27006, 8159, 739, 605, 605, 2, 41732, 11355, 8270, 9383, 41787, 28478, 30613, 8711, 15010, 15354, 8137, 9183, 6824, 7098, 4, 16518, 9208, 18128, 17133, 6824, 2, 41732, 11355, 8270, 9131, 28005, 7235, 9183, 8711, 8240, 9705, 7609, 8285, 8152, 4, 28005, 7692, 7071, 7235, 9183, 15084, 7055, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,

In [27]:
def collate_fn(batch):
    input_ids = [items[0] for items in batch]
    token_type_ids = [items[1] for items in batch]
    labels = [items[2] for items in batch]
    
    return torch.LongTensor(input_ids), torch.LongTensor(token_type_ids), \
            torch.LongTensor(labels)

In [28]:
train_dataloader = DataLoader(train_set, batch_size=Config.batch_size, num_workers=2,
                              shuffle=True, collate_fn=collate_fn)

In [29]:
valid_set = CustomDataset(data_valid, tokenizer, Config)
valid_dataloader = DataLoader(valid_set, batch_size=Config.batch_size, num_workers=2,
                            shuffle=False, collate_fn=collate_fn)

In [30]:
input_id, token_type_id, label = valid_set[0]
print("input_id", input_id, sep='\n')
print("token_type_id", token_type_id, sep='\n')
print("label", label, sep='\n')

input_id
[0, 2, 9443, 29161, 8155, 7172, 10972, 8612, 7584, 8614, 8217, 8711, 7949, 605, 605, 7981, 9021, 7512, 9830, 8705, 8263, 739, 4, 11786, 8101, 739, 605, 605, 7584, 8614, 8163, 8717, 8052, 406, 739, 2, 11018, 9685, 7253, 6866, 7490, 7098, 216, 739, 4, 10152, 406, 739, 216, 27752, 8614, 10132, 9779, 739, 376, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
token_type_id
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

# 5. Model

In [31]:
model = AutoModelForCausalLM.from_pretrained(Config.pretrained_model_name).to(Config.device)

# 6. Train

In [35]:
class Train:
    def __init__(self, model, tokenizer, Config):
        self.model = model
        self.tokenizer = tokenizer
        self.optimizer = torch.optim.AdamW(model.parameters(), lr=Config.learning_rate)
#         self.scheduler = torch.optim.lr_scheduler.ExponentialLR(self.optimizer, gamma=0.9)
        self.bos_token = Config.bos_token
        self.eos_token = Config.eos_token
        self.usr_token = Config.usr_token
        self.sys_token = Config.sys_token
        self.device = Config.device
        self.train_losses = []
        self.valid_losses = []

    def train(self, epochs, train_dataloader, valid_dataloader=None, save=False):
        for epoch in range(epochs):
            print(f"Epoch: {epoch + 1} / {epochs}")
            self.model.train()
            losses = []
            start_time = time.time()

            for i, batch in enumerate(train_dataloader):
                input_ids, token_type_ids, labels = batch        
                input_ids, token_type_ids, labels = \
                    input_ids.to(self.device), token_type_ids.to(self.device), labels.to(self.device)
                
                outputs = self.model(
                    input_ids = input_ids,
                    token_type_ids = token_type_ids,
                    labels = labels
                )
                
                loss = outputs.loss
                
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()

                losses.append(loss.item())
                train_loss = np.mean(losses)
                print(self.status(i + 1, len(train_dataloader), time.time() - start_time,
                                  train_loss), end='\r')
                
#             self.scheduler.step()
            self.train_losses.append(train_loss)
            
            if valid_dataloader:
                valid_loss = self.validation(valid_dataloader)
                print(self.status(i + 1, len(train_dataloader), time.time() - start_time,
                                train_loss) + f" #valid_loss: {valid_loss:.6f}\n", end='\r')
                self.valid_losses.append(valid_loss)
            
            if save:
                time_zone = datetime.timezone(datetime.timedelta(hours=9))
                now = datetime.datetime.now(time_zone)
                PATH = now.strftime(f'models/%m%d_%H%M_ep{epoch + 1}.pt')
                torch.save(self.model.state_dict(), PATH)

    def validation(self, valid_dataloader):
        self.model.eval()
        losses = []
        
        with torch.no_grad():
            for i, batch in enumerate(valid_dataloader):
                input_ids, token_type_ids, labels = batch
                input_ids, token_type_ids, labels = \
                    input_ids.to(self.device), token_type_ids.to(self.device), labels.to(self.device)
                
                outputs = self.model(
                    input_ids = input_ids,
                    token_type_ids = token_type_ids,
                    labels = labels
                )
                
                loss = outputs.loss
                losses.append(loss.item())
            
            valid_loss = np.mean(losses)
        
        return valid_loss
    
    def status(self, step, steps, past_time, train_loss):
        return f"#step: {step} / {steps} #past: {int(past_time)}s #left: {int(steps / step * past_time - past_time)}s #train_loss: {train_loss:.6f}"

    def save(self, PATH=None):
        if not PATH:
            time_zone = datetime.timezone(datetime.timedelta(hours=9))
            now = datetime.datetime.now(time_zone)
            PATH = now.strftime(f'models/%m%d_%H%M_ep{epochs}.pt')
            
        torch.save(self.model.state_dict(), PATH)
        print("model saved.")


In [36]:
chathuman = Train(model, tokenizer, Config)

In [ ]:
chathuman.train(Config.epochs, train_dataloader, valid_dataloader, True)
# chathuman.save(f'models/{Config.model_name}')

In [ ]:
plt.plot(chathuman.train_losses)
plt.plot(chathuman.valid_losses)
plt.show()